# Scattering Amplitude Reconstruction in Python

###### Giuseppe De Laurentis - University of Edinburgh

[PyHEP indico]("https://indico.cern.ch/event/1566263/timetable/#10-scattering-amplitude-recons")

## Motivation

Next-to-next-to-leading order predictions are essential to match the experimental precision

<img src="1911.00479.crosssection.png" style="max-width:150mm; margin-top:2mm; margin-bottom:0mm;">
<center> $\sigma_{pp\rightarrow \gamma\gamma\gamma}$ Chawdhry et al. <a href=https://arxiv.org/abs/1911.00479> arXiv:1911.00479 </a> </center>

1) At NNLO (~ two-loop Feynman diagrams) analytic computations are often unfeasible, especially at high multiplicities
2) Even if an analytic result is avaiable, it may be hard to achieve simpler results by analytic manipulations

Simpler results are not just a theorist delight, they are essential for phenomenology

<img src="quad-precision-no-rescue.png" style="max-width:150mm; margin-top:2mm; margin-bottom:0mm;">

<center> Evaluation of the two-loop corrections to pp -> Vjj obtained in 2021 </center>

<img src="h2__g_g__Z_b_b.stability.png" style="max-width:150mm; margin-top:2mm; margin-bottom:0mm;">

## What is analytic reconstruction?

A) Compute numerically to bypass all intermediate complexities <br>
B) Analytise the numerical evaluations to infer the exact analytic representation 

<br>

<center> <b> Analytic reconstruction is an alternative to analytic manipulations </b> </center>

<br>

A) it is needed both to obtain new NNLO corrections <br>
B) and to make available expressions efficient / usable

Efficiency is also a pressing need: the $\alpha_s$extraction from energy-energy correlators in $pp\rightarrow jjj$ by ATLAS [arXiv:2301.09351](https://arxiv.org/abs/2301.09351) used $10^8$CPUh

## Why Python over Mathematica?

Cons:
1) analytic manipulations are more awkward - but we will do mostly numerical work

Pros:
1) Name spaces: variables in Python are local by default, in Mathematica they are global
2) Object oriented: in Python we can attach methods and properties to objects, in Mathematica everything is a glorified list
3) Package manager (PyPI), in Mathematica pass files by email or attach to papers <br>
3b. Large community with many available packages
4) Continuous integration: pytest, flint, etc. - in Mathemtica, write a Python script that runs the test
5) Licenses: multi-processing / threading not limited by number of bought licences
6) Open source

Is performance an issue? Not really, outsource compute intensive operations to C++ / CUDA / Fortran / Rust (interfacing is simple)!

## A toy example

Work with spinor-helicity variables:
$$p_{i, \mu} \sigma_\mu^{\dot\alpha\alpha} = p^{\dot\alpha\alpha}_i = \tilde\lambda_i^{\dot\alpha}\lambda_i^{\alpha} = |i]\langle i | \; \text{ if } \;  p^2 = 0 $$
and Mandelstam invariants:
$$s_{ij\dots k} = (p_i + p_j + \dots + p_k)^2 $$

In [13]:
mandelstam_expression = "(1/(⟨14⟩^2⟨15⟩^2⟨23⟩^2))⟨12⟩^3⟨13⟩((4s23(-(s23s34+(s15-s34)s45)^3(s23s34+s45(s15+s34+s45))+s12^3(s15-s23)(s15^3s45+s23^2s34(-s23+s45)+s15^2s45(-s23+s45)+s15(s23^2s34-s23s45^2-s34s45^2))-s12^2(3s15^4s45^2+s15^3s45^2(-4s23-2s34+3s45)+s23s34^2(3s23^3-4s23^2s45+s45^3)+s15^2(-s23s45^2(s34+4s45)-s34s45^2(s34+5s45)+s23^2(s34^2+s45^2))+s15(-4s23^3s34^2+2s34^2s45^3+s23s34s45^2(s34+2s45)+s23^2s45(s34^2+s45^2)))+s12(3s15^4s45^3+s15^3s45^2(4s23s34-2s23s45-4s34s45+3s45^2)+s34^2(s23-s45)^2(3s23^2s34-s34s45^2+s23s45(s34+s45))-s15^2s45(s23^2s34(s34+s45)+s34s45^2(s34+7s45)+2s23s45(2s34^2-s34s45+s45^2))-s15s34(s23-s45)(2s23^2s34(s34-2s45)+s34s45^2(2s34+5s45)+s23s45(2s34^2+2s34s45+s45^2)))))/(3s12^3(s15-s23)s34(s12+s23-s45)s45(s15+s45)(-s12+s34+s45))+(4s23((s23s34+(s15-s34)s45)^2(s23s34+s45(s15+s34+s45))+s12^2(s23^2s34(s23-s45)+s15^3s45+s15^2s45(-s23+s45)-s15(s23^2s34+s23s45^2+s34s45^2))+s12(-2s15^3s45^2+s34^2(-2s23^3+2s23^2s45+s23s45^2-s45^3)+s15^2s45((s34-2s45)s45+s23(-s34+s45))+s15(s23^2s34(s34-s45)+s23s45^3+s34s45^2(s34+3s45))))(-tr5_1234))/(3s12^3(s15-s23)s34(s12+s23-s45)(s12-s34-s45)s45(s15+s45)))[31]"

def black_box_function(phase_space_point):  # may be a complicated expression we want to simplify, or a numerical routine
    return phase_space_point(mandelstam_expression)

In [2]:
from antares import Terms  # pip install antares-hep (Automated Numerical To Analytical Reconstruction Software)

In [3]:
# Goal: reconstruct the simplest possible form of this function
oTerms = Terms("""
+(8/3s23⟨24⟩[34])/(⟨15⟩⟨34⟩⟨45⟩⟨4|1+5|4])
+('12354', False, '+')
""")

### Flash overview of lips

In [14]:
from lips import Particles  # pip install lips (Lorentz invariant phase space)

In [5]:
# random (complex) phase space point with 5 massless legs, default is complex, 300 digits of precision
# uses mpmath for the arbitrary precision floating-point numbers (mpc and mpf)
oPs = Particles(5)

In [6]:
oPs[1].r_sp_d  # right-handed spinor index down

array([[mpc(real='1.24195077849123031272032599508542317615688597687529085037004195107345940549107374383187541823255949714991498494294544976945489967523327943053960147895422419762361945277640703451332567148759949819745103568890058964036115610221434981333178486399294644397204502956918145363910773324739987416781690332529566', imag='-0.0500121289515306672937206435213243520469004303051673485931758453477226522193713110998547139805408531196909149907472511605008674752360957262365145889204347530022183967895130739474333568419626520992674566292617264943748621653356549455126104152455573002624929163803495096070128881460601611813274810154609348')],
       [mpc(real='-0.21598799719175152906913932321130103307604998794809613770790085325011740044548993632840438404514615972029021535937890811135132474453508277678671106533319061123767530438020667493126307187077678296025013425475577657851720899284961910195562628619921794590517306617648939398079855088827455240556199161948833', imag='0.85784827690404787330625

i.e. $\tilde\lambda_{1, \dot\alpha}$

In [7]:
oPs[1].l_sp_d  # left-handed spinor index down

array([[mpc(real='1.24195077849123031272032599508542317615688597687529085037004195107345940549107374383187541823255949714991498494294544976945489967523327943053960147895422419762361945277640703451332567148759949819745103568890058964036115610221434981333178486399294644397204502956918145363910773324739987416781690332529566', imag='-0.0500121289515306672937206435213243520469004303051673485931758453477226522193713110998547139805408531196909149907472511605008674752360957262365145889204347530022183967895130739474333568419626520992674566292617264943748621653356549455126104152455573002624929163803495096070128881460601611813274810154609348'),
        mpc(real='-0.0417585035343757130162327191452397357522288220146157016390548425202651431598735478223077872864925957353495347183631708411431588153234925291864789947511266789880328385396804757487043565050647829242898966108696327656863785193600854265100888106739286173696405751567279622384345593390959973604296620557718894', imag='-0.789028959756538574072

i.e. $\lambda_{1, \alpha}$

In [8]:
oPs[1].four_mom # p^\mu

array([mpc(real='1.1129134961256814633720159734877182470639643892640540691056887606342781012946862215404922774622487487122389104804370818335320473822991253224333049714327377382430423117797800547760333230686131261633750014261227156546204013373822996474394674598706813835892837213531202630261750985837216077451149804997537', imag='0.0051865597351677323792460968650675829864758524643789774434841590191864502745529300571373864085557279128899784227219875245765131122126195826035522460416529146089440238594845620975427864165057353051145376490765100504102754852866764826137435091815082233220954939456840039569588209530804846085680425877482734'),
       mpc(real='-0.15833333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333', imag='0.0491803278688524590163934426

Implements getters and setters, e.g. updating the value of .four_mom will automatically update the spinors (and vice versa)

In [17]:
# I've implemented a custom ast (abstract syntactic tree) parser (this is hidden behind Particles.__call__)
oPs._parse(mandelstam_expression)

"(1/(oPs('⟨1|4⟩')**2*oPs('⟨1|5⟩')**2*oPs('⟨2|3⟩')**2))*oPs('⟨1|2⟩')**3*oPs('⟨1|3⟩')*((4*oPs('s_23')*(-(oPs('s_23')*oPs('s_34')+(oPs('s_15')-oPs('s_34'))*oPs('s_45'))**3*(oPs('s_23')*oPs('s_34')+oPs('s_45')*(oPs('s_15')+oPs('s_34')+oPs('s_45')))+oPs('s_12')**3*(oPs('s_15')-oPs('s_23'))*(oPs('s_15')**3*oPs('s_45')+oPs('s_23')**2*oPs('s_34')*(-oPs('s_23')+oPs('s_45'))+oPs('s_15')**2*oPs('s_45')*(-oPs('s_23')+oPs('s_45'))+oPs('s_15')*(oPs('s_23')**2*oPs('s_34')-oPs('s_23')*oPs('s_45')**2-oPs('s_34')*oPs('s_45')**2))-oPs('s_12')**2*(3*oPs('s_15')**4*oPs('s_45')**2+oPs('s_15')**3*oPs('s_45')**2*(-4*oPs('s_23')-2*oPs('s_34')+3*oPs('s_45'))+oPs('s_23')*oPs('s_34')**2*(3*oPs('s_23')**3-4*oPs('s_23')**2*oPs('s_45')+oPs('s_45')**3)+oPs('s_15')**2*(-oPs('s_23')*oPs('s_45')**2*(oPs('s_34')+4*oPs('s_45'))-oPs('s_34')*oPs('s_45')**2*(oPs('s_34')+5*oPs('s_45'))+oPs('s_23')**2*(oPs('s_34')**2+oPs('s_45')**2))+oPs('s_15')*(-4*oPs('s_23')**3*oPs('s_34')**2+2*oPs('s_34')**2*oPs('s_45')**3+oPs('s_23')*oPs(

In [18]:
complex(oTerms(oPs) - oPs(mandelstam_expression)) # evaluation of Terms object is optimized (including upcoming GPU support)

(-2.8931172173599785e-300-2.2398326844077253e-300j)

In [10]:
expr = "+(8/3s23⟨24⟩[34])/(⟨15⟩⟨34⟩⟨45⟩⟨4|1+5|4])"
complex((oPs(expr) + oPs.image(('12354', False))(expr)) - oTerms(oPs))

1.4932217896051502e-300j

In [46]:
# this generates a "real" phase space point (all outgoing convention, energy might be negative)
oPs = Particles(5, real_momenta=True, seed=0)

In [47]:
oPs[1].four_mom

array([mpc(real='0.687332511', imag='0.0'),
       mpc(real='-0.0102564096', imag='0.0'),
       mpc(real='0.636363626', imag='0.0'),
       mpc(real='-0.259541988', imag='0.0')], dtype=object)

### Main issue: numerical stability

In [48]:
from syngular import Field  # pip install syngular (interface and extension to Singular, https://www.singular.uni-kl.de/)

Simulate a long running computation where error accumulates by lowering the precision

In [49]:
oPs = Particles(5, field=Field("mpc", 0, 6), seed=0)

In [50]:
abs(oTerms(oPs) - black_box_function(oPs))

mpf('2.38776875e-6')

In [51]:
oPs = Particles(5, field=Field("mpc", 0, 6), seed=2)

In [52]:
abs(oTerms(oPs) - black_box_function(oPs))  # lost precision! :'(

mpf('0.428962946')

In [53]:
black_box_function(oPs)

mpc(real='-1.13071203', imag='25.7996826')

In [54]:
oTerms(oPs)

mpc(real='-0.711602092', imag='25.8910942')

In [55]:
oPs = Particles(5, field=Field("mpc", 0, 16), seed=2)

In [56]:
black_box_function(oPs)

mpc(real='-0.7116312918164890561', imag='25.89107035069941443')

In [57]:
oTerms(oPs)

mpc(real='-0.7116312918068779803', imag='25.89107035076286678')

## finite fields

$$ 
\displaystyle a \in \mathbb{F}_p : a \in \{0, \dots, p -1\} \; \text{ with } \; \{+, -, \times, \div\}
$$

In [58]:
from fractions import Fraction as Q
from pyadic import ModP

In [59]:
ModP(Q(1, 2), 11)

6 % 11

or

In [60]:
from syngular import Field

In [61]:
Fp = Field("finite field", 2 ** 31 - 1, 1)
Fp

Field('finite field', 2147483647, 1)

In [62]:
Fp.random()

80909179 % 2147483647

what goes wrong if p is not prime?

In [63]:
ModP(Q(1, 2), 6)

ZeroDivisionError: Inverse of 2 % 6 mod 6 does not exist. Are you sure 6 is prime?

the inverse cannot be computed for any number not co-prime with the chosen modulus

### Phase space in a finite field

In [64]:
oPs = Particles(5, field=Field("finite field", 2 ** 31 - 1, 1), seed=0)

In [65]:
black_box_function(oPs)

497973027 % 2147483647

In [66]:
oTerms(oPs)

497973027 % 2147483647

### The trivial absoulute value

$$ |a|_0 = 0 \; \text{if} \; a = 0 \; \text{else} \; 1$$

In [71]:
abs(ModP(Q(2), 5))

1

In [72]:
abs(ModP(Q(0), 5))

0

### Limitation

a) can make things zero, but not small <br>
b) cannot take limits <br>
c) calculus is undefined

In [41]:
Fp.epsilon()

ValueError: Finite field infinitesimal does not exist.

### [Ostrowski theorem](https://en.wikipedia.org/wiki/Ostrowski%27s_theorem)

There exist 3 possible absolute values on the rationals: <br>
     - the trivial absolute value $|\cdot|_0$ <br>
     - the real absolute value $|\cdot|_\infty$ <br>
     - the p-adic absolute value $|\cdot|_p$

## p-adic numbers

In [10]:
from pyadic import PAdic

In [11]:
PAdic(Q(1, 2), 11, 3)

6 + 5*11 + 5*11^2 + O(11^3)

In [34]:
Qp = Field("padic", 2 ** 31 - 1, 11)
Qp

Field('padic', 2147483647, 11)

In [35]:
Qp.random()

1987708681 + 2098470235*2147483647 + 98393051*2147483647^2 + 417276484*2147483647^3 + 1629737080*2147483647^4 + 771229528*2147483647^5 + 1045128399*2147483647^6 + 1932607596*2147483647^7 + 1385764091*2147483647^8 + 567178084*2147483647^9 + 1893123388*2147483647^10 + O(2147483647^11)

In [36]:
Qp.epsilon()

2147483647

## Interpolation algorithms

In [74]:
from pyadic.interpolation import Newton_polynomial_interpolation, Thiele_rational_interpolation

### Why Reconstruction $\supset$ Interpolation

## Least Common Denominators

In [34]:
field = Field("finite field", 2 ** 31 - 1, 1)
seed = 0 
oSliceFp = Particles(5, field=field, seed=seed)
oSliceFp.univariate_slice(algorithm='covariant', seed=seed)

In [38]:
from antares import settings
from antares.core.numerical_methods import num_func

In [41]:
settings.invariants = Invariants(5).full

In [42]:
black_box_function.multiplicity = 5
oF = num_func(black_box_function)

In [44]:
oF.get_lcd(oSliceFp, verbose=True)

Finished after 19 samples, [162860843 % 2147483647, 1885044667 % 2147483647, 1411810336 % 2147483647, 1402519729 % 2147483647, 468240741 % 2147483647, 1197585802 % 2147483647, 1947316210 % 2147483647, 291407823 % 2147483647, 450073212 % 2147483647, 882656524 % 2147483647, 1372881572 % 2147483647, 934808415 % 2147483647, 1385619189 % 2147483647, 1938318280 % 2147483647, 1622103845 % 2147483647, 869915779 % 2147483647, 1364220815 % 2147483647, 64322534 % 2147483647, 552291472 % 2147483647]. 
 (-514009628*t**8 + 776677322*t**7 + 991846540*t**6 - 75374069*t**5 + 101225610*t**4 + 36666516*t**3 - 728369354*t**2 + 534567587*t - 70010385)/(t**9 - 49350670*t**8 + 1034569225*t**7 + 606286545*t**6 + 418505031*t**5 - 958181787*t**4 - 882182274*t**3 - 369000092*t**2 - 291896180*t - 698421050)
Polynomial defaultdict(<class 'int'>, {t - 505232661: 1, t - 179967131: 1, t + 594877252: 1, t + 969664725: 1, t**4 + 449954863*t**3 + 382707815*t**2 - 580761694*t - 422746403: 1})
Matched 2 / 8: {'⟨2|3⟩': 1, 

Terms("""+(1⟨2|3⟩[2|3])/(⟨1|4⟩⟨1|5⟩⟨3|4⟩⟨3|5⟩⟨4|5⟩⟨4|(1+5)|4]⟨5|(1+4)|5])""")

In [45]:
oTerms.get_lcd(oSliceFp)

Terms("""+(1⟨2|3⟩[2|3])/(⟨1|4⟩⟨1|5⟩⟨3|4⟩⟨3|5⟩⟨4|5⟩⟨4|(1+5)|4]⟨5|(1+4)|5])""")

## Computational Algebraic Geometry

## Looking forward

In [54]:
from antares.core.unknown import Unknown
from antares.numerical_to_analytical import numerical_to_analytical

In [58]:
oF.do_single_collinear_limits()
oF.do_double_collinear_limits()

The partial result is:                                                                                             
(1⟨2|3⟩[2|3])/(⟨1|4⟩⟨1|5⟩⟨3|4⟩⟨3|5⟩⟨4|5⟩⟨4|(1+5)|4]⟨5|(1+4)|5])

Mass dimension & phase weights: -1.0, [-1, 1, -2, -2, -2] → 6.0, [1, 1, 0, 1, 1]
Cleaning pair scalings results from known numerator information.                                                             
Finished calculating pair scalings. They are:                         
[⟨1|4⟩, ⟨1|5⟩]:             3.0, 31 → 7
[⟨1|4⟩, ⟨3|4⟩]:             3.0, 26 → 7
[⟨1|4⟩, ⟨3|5⟩]:             2.0, 2  → 2
[⟨1|4⟩, ⟨4|5⟩]:             3.0, 31 → 7
[⟨1|4⟩, ⟨4|(1+5)|4]]:       1.0, 3  → 2
[⟨1|4⟩, ⟨5|(1+4)|5]]:       2.0, 4  → 2
[⟨1|5⟩, ⟨3|4⟩]:             2.0, 4  → 2
[⟨1|5⟩, ⟨3|5⟩]:             3.0, 26 → 7
[⟨1|5⟩, ⟨4|5⟩]:             3.0, 31 → 7
[⟨1|5⟩, ⟨4|(1+5)|4]]:       2.0, 3  → 2
[⟨1|5⟩, ⟨5|(1+4)|5]]:       1.0, 3  → 2
[⟨3|4⟩, ⟨3|5⟩]:             3.0, 31 → 7
[⟨3|4⟩, ⟨4|5⟩]:             3.0, 31 → 7
[⟨3|4⟩, ⟨4|(1+5)|4]]:

In [59]:
oU = Unknown(oF)

In [60]:
numerical_to_analytical(oU)

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
NoneToAnalytical called at depth 0.

The partial result is:                                                                                             
(1⟨2|3⟩[2|3])/(⟨1|4⟩⟨1|5⟩⟨3|4⟩⟨3|5⟩⟨4|5⟩⟨4|(1+5)|4]⟨5|(1+4)|5])

Mass dimension & phase weights: -1.0, [-1, 1, -2, -2, -2] → 6.0, [1, 1, 0, 1, 1]
Cleaning pair scalings results from known numerator information.                                                             
Finished calculating pair scalings. They are:                         
[⟨1|4⟩, ⟨1|5⟩]:             3.0 (3.0), 31 → 7
[⟨1|4⟩, ⟨3|4⟩]:             3.0 (3.0), 26 → 7
[⟨1|4⟩, ⟨3|5⟩]:             2.0 (2.0), 2  → 2
[⟨1|4⟩, ⟨4|5⟩]:             3.0 (3.0), 31 → 7
[⟨1|4⟩, ⟨4|(1+5)|4]]:       1.0 (1.0), 3  → 2
[⟨1|4⟩, ⟨5|(1+4)|5]]:       2.0 (2.0), 4  → 2
[⟨1|5⟩, ⟨3|4⟩]:             2.0 (2.0), 4  → 2
[⟨1|5⟩, ⟨3|5⟩]:             3.0 (3.0), 26 → 7
[⟨1|5⟩, ⟨4|5⟩]:             3.0 (3.0), 31 → 7
[⟨1|

[Terms("""+[2|3]⟨2|3⟩(8/3⟨2|5⟩⟨1|4⟩⟨2|4⟩⟨3|5⟩[2|3][4|5]-8/3⟨4|5⟩⟨2|5⟩⟨2|4⟩[3|5]⟨1|3⟩[2|4]-16/3⟨4|5⟩⟨2|4⟩⟨3|5⟩[3|5]⟨1|3⟩[3|4]+8/3⟨4|5⟩⟨1|2⟩⟨3|5⟩⟨3|4⟩[3|5][3|4]+8/3⟨4|5⟩²[3|5]⟨1|3⟩⟨2|3⟩[3|4])/(⟨1|4⟩⟨1|5⟩⟨3|4⟩⟨3|5⟩⟨4|5⟩⟨4|(1+5)|4]⟨5|(1+4)|5])""")]

## GitHub self hosted runner (for GPU access)